# 勉強内容

## Gain Map Specification を読んで (1.0 draft 12, May 2023)

### Concept

* ファイルに保存するのは "Base" rendition と "Gain Map"
* Base は SDR でも HDR でも OK
  * Base が SDR なら Gain Map で HDRを作る
  * Base が HDR なら Inverse Gain Map で SDRを作る
* SDR を Base にすると後方互換性を担保できる
* Gain Map をいい感じに interpolate できるのは柔軟性があってよい

### How to Compute the Gain Map

* SDR と HDR の画像は事前に作成済みとする
  * SDR と HDR の画像生成方法は本仕様書のスコープ外
* SDR と HDR の画像はそれぞれ Linear であり、かつ同じ primary を持つものとする（白色点は？）

#### Step

1. [Option] lower resolution にダウンサンプルする
2. [Option] グレイスケールに変換する
3. $G = \log_2\left( \frac{HDR + K_{hdr}}{SDR + K_{sdr}} \right)$

  * 上記の式で 1面または 3面の logの Gain Map を得る
  * $K_{hdr}$ と $K_{sdr}$ は計算を破綻させないための係数
  * 分母と分子は正の数であること

4. $G$ の minimum, mean, maximum を各色ごとに求める
5. 各色の G を [0.0, 1.0] に正規化する
6. [Option] OETF 的な Gammaカーブを適用
7. 量子化する。8-bit とか 10-bit とかに
8. Gain Map のメタデータ計算してファイルに添付
9. [Option] Gain Map を圧縮する

#### Gain Map Metadata

以下のデータを Gain Map Image に加えて保存する

* 各チャネル毎の min, max の Gain Max value
* 各チャネル毎の Gamma値（逆に言うと Gamma はチャネル毎に変更可能）
* 各チャネル毎の $K_{hdr}$ と $K_{sdr}$
* Minimum, Maximum の HDR capacity value (Gain Map 補間用)
  * これも log2 ベース
* Base rendition が SDR か HDR かを示す boolean flag
* バージョンタグ

#### メモ

\begin{align*}
G &= \log_2\left( \frac{HDR + K_{hdr}}{SDR + K_{sdr}} \right) \\
2^G &= \frac{HDR + K_{hdr}}{SDR + K_{sdr}} \\
(SDR + K_{sdr})\cdot 2^G &= HDR + K_{hdr} \\
HDR &= (SDR + K_{sdr})\cdot 2^G - K_{hdr}
\end{align*}


#### How to Apply the Gain Map

1. Base rendition in linear gamma から始める。例えば P3, Gamma 2.2 を解除した画像
2. Gain Map と関連する metadata を読む。Gain Map は整数型から [0.0, 1.0] の浮動小数点に戻す
3. Gamma適用、min/max を使ったスケール、$K_{hdr}$ と $K_{sdr}$ のオフセット適用を順に行い、元の $G$ を復元する
4. 元の画像に合うように Gain Map をリサンプルする。リサンプルのアルゴリズムは任意だが Nearest Neighbor は非推奨
5. ウェイト $W$ を使って Gain Map をスケールする
6. $2^G$ をして Gain Map を Linear に戻す
7. Gain Map がグレイスケールだった場合は R, G, B の各色に適用する
X. その後で PQ とか HLG とかでエンコードする

#### How to Scale the Gain Map

* Gain Map をスケールする目的は表示デバイス側の HDR capacity に応じて SDR と HDR の間を作ること
* スケールの際はウェイト $W$ を乗算する
  * 乗算は Log空間上で行う（人間の感覚は Log だし）
* $W$ は以下の手順で計算する
  * $H$ を求める
    * $H$ は HDR と SDR white の比に対して $\log_2$ を計算したもの
    * 例えば表示デバイス側のピーク輝度が SDR white と同じだったら $H = \log_2 (203/203) = 0$ となる
  * $M_{lo}$ と $M_{hi}$ はそれぞれ Gain Map での最小・最大の値を意味する。

* これが推奨の実装である。ベースが SDR の場合。
* W の計算方法は以下

\begin{equation*}
\begin{align*}
H &= \log_2 \left(\frac{HDR_{white}}{SDR_{white}}\right) \\
F &= clamp \left(\frac{H - M_{lo}}{M_{hi} - M_{lo}}, 0, 1\right)
\end{align*}
\end{equation*}

* SDR がベースなら $W = F$
* HDR がベースなら $W = F - 1$

* 以下に注意事項？を挙げる
  * $H \leq M_{lo}$ であれば結果は SDR になる
  * $H \geq M_{hi}$ であれば結果は HDR になる
  * $M_{lo} < H < H_{hi}$ の時は SDR と HDR の中間になる

#### Recommended Usage of Minimum and Maximum HDR Capacity

* Lightroom とかでレタッチする際、HDR capacity は +3 程度に納めておくべき。
* +6 とかにすると 203 * 2^6 = 12,992 nits がモニター側の最大ピーク輝度になってしまうから
